In [1]:
import requests
import base64
from urllib.parse import urlencode
import json
import datetime

#import constants from KEYS module. create a module called KEYS.py and Add your own Client ID and Client Secret to it
import KEYS

In [2]:
requests.__version__

'2.25.1'

In [3]:
#assign keys to values here

client_id = KEYS.CLIENT_ID
client_secret = KEYS.CLIENT_SECRET

In [4]:
#format and encode credential in to expected form for api

client_creds = f"{client_id}:{client_secret}"
client_creds_b64 = base64.b64encode(client_creds.encode())
print(base64.b64decode(client_creds_b64).decode())
print(client_creds)
print(client_creds_b64.decode())

af37f76b6c204b16b5f781ce1e54603d:cdc8546a2bdd42ea96f3fe6a860ac1d3
af37f76b6c204b16b5f781ce1e54603d:cdc8546a2bdd42ea96f3fe6a860ac1d3
YWYzN2Y3NmI2YzIwNGIxNmI1Zjc4MWNlMWU1NDYwM2Q6Y2RjODU0NmEyYmRkNDJlYTk2ZjNmZTZhODYwYWMxZDM=


In [5]:
#Builds the parts of the querey
token_url = "https://accounts.spotify.com/api/token"
method = "POST"
token_data = {
    "grant_type":"client_credentials",
}
token_headers = {
    "Authorization" : f"Basic {client_creds_b64.decode()}",
}

In [6]:
#makes GET request to spotify API
r = requests.post(token_url, headers=token_headers, data=token_data)

#validates result, then prints
valid_request = r.status_code in range(200,299)
print(r.json())

{'access_token': 'BQD0r0eyl-C75DV2WcK3OCi6wTkYit_K3HEMMiueCAeoyKVoIGrq4kA7ZhN9fhUoedoWt6IPQL2UrY9Oshw', 'token_type': 'Bearer', 'expires_in': 3600}


In [7]:
#uses validation boolean to decide if to parse
if valid_request:
    token_response_data = r.json() #convert json to dict
    now = datetime.datetime.now() # create datetime obj for now
    access_token = token_response_data['access_token'] #retrieve token from response
    expires_in = token_response_data['expires_in'] #retrieve token lifetime
    expires = now + datetime.timedelta(seconds=expires_in) #creates datetime for when token expires
    did_expire = expires < datetime.datetime.now() #boolean to rep if expired

In [8]:
did_expire

False

In [9]:
expires

datetime.datetime(2021, 4, 15, 16, 31, 12, 655671)